In [1]:
%load_ext autoreload

In [2]:
%autoreload
import sys
sys.path.insert(0, '../')
sys.path.insert(0, '../../holmes-evaluation/src/')

In [3]:
from src.utilities.mluar_utils import *

/mnt/swordfish-pool2/milad/conda-envs/huggingface-tlr/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset, Dataset
import numpy as np
from einops import rearrange, reduce, repeat
import torch
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import pyplot as plt
import math

In [5]:
MULTI_LUAR_PATH =  "/mnt/swordfish-pool2/milad/multi-luar-reddit-model/"
LUAR_PATH =  "/mnt/swordfish-pool2/nikhil/LUAR/pretrained_weights/LUAR-MUD/"

In [6]:
from sentence_transformers.models import Transformer
from model.ParallelSentenceTransformer import ParallelSentenceTransformer
from utils.SpecificLayerPooling import MultiLUARsPooling

In [7]:
transformer = Transformer(MULTI_LUAR_PATH, model_args={'trust_remote_code':True}, config_args={'sentence_transformer_support':True, 
                                                                                               'trust_remote_code':True,
                                                                                               'output_hidden_states':True})
base_model = ParallelSentenceTransformer(modules=[transformer, MultiLUARsPooling(word_embedding_dimension=transformer.get_word_embedding_dimension(), layers=[0])])
base_model = base_model.to('cuda')

In [8]:
res = base_model.encode(
    sentences=['hi mister', 'glad to see you'], show_progress_bar=True, batch_size=4,
    output_value='sentence_layer_embeddings', convert_to_numpy=True, device='cuda'
)

Batches: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.32it/s]


In [9]:
res[0].shape

(2, 512)